In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import matplotlib as mpl 
mpl.rcParams['figure.dpi'] = 400 # figure dots per inch
%matplotlib inline

import seaborn as sns
sns.set()

from datetime import datetime, timedelta

In [3]:
# Load transactions (all data including March)  
transactions_df = pd.read_pickle('Data/transactions_clean.pickle')
transactions_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dt,membership_expire_date_dt
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31
10,bQkbrEPdMfVfdsoz0RoWSmgYpHJuhRqtIml7aRUrFL8=,39,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21
24,qNpZJ7gsMPAFSvc5V3kgqtFfUQU2SZe7XSUYOAfqtoA=,40,30,149,149,1,20151001,20151101,0,2015-10-01,2015-11-01
26,WnxdSgeK3Vg4BIaioJQYilnL+E1SDxq+PZIwmmfTwgM=,34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31
27,ir1jh/vWZ932FpSC2WeMxdOHQ+OkuUo3ZPQs8Ms4J9g=,34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31


In [4]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13664286 entries, 1 to 22974713
Data columns (total 11 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   msno                       object        
 1   payment_method_id          int16         
 2   payment_plan_days          int16         
 3   plan_list_price            int16         
 4   actual_amount_paid         int16         
 5   is_auto_renew              int16         
 6   transaction_date           int64         
 7   membership_expire_date     int64         
 8   is_cancel                  int16         
 9   transaction_date_dt        datetime64[ns]
 10  membership_expire_date_dt  datetime64[ns]
dtypes: datetime64[ns](2), int16(6), int64(2), object(1)
memory usage: 781.9+ MB


In [5]:
day_dict = {0: 'Mon',
            1: 'Tue',
            2: 'Wed',
            3: 'Thu',
            4: 'Fri',
            5: 'Sat',
            6: 'Sun'}
month_dict = {1: 'Jan',
             2: 'Feb',
             3: 'Mar',
             4: 'Apr',
             5: 'May',
             6: 'Jun',
             7: 'Jul',
             8: 'Aug',
             9: 'Sep',
             10: 'Oct',
             11: 'Nov',
             12: 'Dec'}

transactions_df['transaction_date_day'] = transactions_df['transaction_date_dt'].dt.weekday.map(day_dict)
transactions_df['membership_expire_date_day'] = transactions_df['membership_expire_date_dt'].dt.weekday.map(day_dict)

transactions_df['transaction_date_month'] = transactions_df['transaction_date_dt'].dt.month.map(month_dict)
transactions_df['membership_expire_date_month'] = transactions_df['membership_expire_date_dt'].dt.month.map(month_dict)

transactions_df['transaction_date_year'] = transactions_df['transaction_date_dt'].dt.year
transactions_df['membership_expire_date_year'] = transactions_df['membership_expire_date_dt'].dt.year

In [6]:
transactions_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dt,membership_expire_date_dt,transaction_date_day,membership_expire_date_day,transaction_date_month,membership_expire_date_month,transaction_date_year,membership_expire_date_year
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015
10,bQkbrEPdMfVfdsoz0RoWSmgYpHJuhRqtIml7aRUrFL8=,39,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,Wed,Sat,Sep,Nov,2015,2015
24,qNpZJ7gsMPAFSvc5V3kgqtFfUQU2SZe7XSUYOAfqtoA=,40,30,149,149,1,20151001,20151101,0,2015-10-01,2015-11-01,Thu,Sun,Oct,Nov,2015,2015
26,WnxdSgeK3Vg4BIaioJQYilnL+E1SDxq+PZIwmmfTwgM=,34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015
27,ir1jh/vWZ932FpSC2WeMxdOHQ+OkuUo3ZPQs8Ms4J9g=,34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015


Let's reduce high cardinality features:

In [7]:
payment_id_freq = transactions_df['payment_method_id'].value_counts(normalize=True)
payment_id_freq.round(4)

41    0.6177
40    0.1105
38    0.0601
37    0.0536
34    0.0439
36    0.0399
33    0.0199
31    0.0147
39    0.0102
30    0.0062
29    0.0050
27    0.0030
35    0.0030
32    0.0026
28    0.0024
23    0.0021
18    0.0009
19    0.0008
14    0.0007
20    0.0007
16    0.0005
24    0.0004
22    0.0003
25    0.0003
26    0.0001
11    0.0001
13    0.0001
21    0.0001
17    0.0001
10    0.0001
12    0.0001
15    0.0000
8     0.0000
5     0.0000
6     0.0000
3     0.0000
2     0.0000
4     0.0000
Name: payment_method_id, dtype: float64

In [8]:
# 80-20 rule - reduce cardinality
bottom_payment_id = payment_id_freq.quantile(q=0.8)
less_freq_payment_id = payment_id_freq[payment_id_freq <= bottom_payment_id]
less_freq_payment_id.round(4)

39    0.0102
30    0.0062
29    0.0050
27    0.0030
35    0.0030
32    0.0026
28    0.0024
23    0.0021
18    0.0009
19    0.0008
14    0.0007
20    0.0007
16    0.0005
24    0.0004
22    0.0003
25    0.0003
26    0.0001
11    0.0001
13    0.0001
21    0.0001
17    0.0001
10    0.0001
12    0.0001
15    0.0000
8     0.0000
5     0.0000
6     0.0000
3     0.0000
2     0.0000
4     0.0000
Name: payment_method_id, dtype: float64

In [9]:
transactions_df.loc[transactions_df['payment_method_id'].isin(less_freq_payment_id.index.tolist()), 'payment_method_id'] = 'other'
transactions_df['payment_method_id'] = transactions_df['payment_method_id'].apply(lambda x: "{}{}".format('payment_method_id_', x))
transactions_df['payment_method_id']

1              payment_method_id_41
10          payment_method_id_other
24             payment_method_id_40
26             payment_method_id_34
27             payment_method_id_34
                     ...           
22962291       payment_method_id_36
22965803       payment_method_id_40
22965957       payment_method_id_40
22967796       payment_method_id_36
22974713       payment_method_id_40
Name: payment_method_id, Length: 13664286, dtype: object

Let's also add some features:

In [10]:
transactions_df['plan_duration'] = (transactions_df['membership_expire_date_dt'] - 
                                  transactions_df['transaction_date_dt'])

transactions_df['plan_duration'] = transactions_df['plan_duration'].astype('timedelta64[D]')
transactions_df['plan_duration'].head()

1     31.0
10    52.0
24    31.0
26    31.0
27    31.0
Name: plan_duration, dtype: float64

In [11]:
transactions_df['plan_duration'].value_counts()

 31.0     5416123
 30.0     4778706
 29.0     1023662
 32.0      900127
 28.0      289981
           ...   
-54.0           1
-118.0          1
-112.0          1
-127.0          1
-235.0          1
Name: plan_duration, Length: 1070, dtype: int64

In [12]:
transactions_df[transactions_df['plan_duration'] < 0].head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dt,membership_expire_date_dt,transaction_date_day,membership_expire_date_day,transaction_date_month,membership_expire_date_month,transaction_date_year,membership_expire_date_year,plan_duration
422,Xik55GiZKTkU1fMHNuROOSIRjf67/9HQ9xYn7Ltldmk=,payment_method_id_other,30,149,149,1,20160225,20160224,1,2016-02-25,2016-02-24,Thu,Wed,Feb,Feb,2016,2016,-1.0
482,ZcPtzHBvgbuQHg3Gx9O4+QX4LK+35NXZSyEUcIbL134=,payment_method_id_37,30,149,149,1,20160826,20160825,1,2016-08-26,2016-08-25,Fri,Thu,Aug,Aug,2016,2016,-1.0
505,q7zVHl6rWbLoKUj3FCHSjO2caWkhhKskVfxPRyLBOHo=,payment_method_id_36,30,180,180,1,20160908,20160907,1,2016-09-08,2016-09-07,Thu,Wed,Sep,Sep,2016,2016,-1.0
513,iEqhCo9ehvlVGXTEm02F4oSarbJUa3IWP13gRD7BFL0=,payment_method_id_36,30,180,180,1,20170211,20170210,1,2017-02-11,2017-02-10,Sat,Fri,Feb,Feb,2017,2017,-1.0
550,sj2G673Oo9mUgQGg1zsN6Cxx6YmSWurMqaoNrsJ50DU=,payment_method_id_40,30,149,149,1,20161225,20161224,1,2016-12-25,2016-12-24,Sun,Sat,Dec,Dec,2016,2016,-1.0


In [13]:
transactions_df[transactions_df['plan_duration'] > 0].head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dt,membership_expire_date_dt,transaction_date_day,membership_expire_date_day,transaction_date_month,membership_expire_date_month,transaction_date_year,membership_expire_date_year,plan_duration
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,payment_method_id_41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015,31.0
10,bQkbrEPdMfVfdsoz0RoWSmgYpHJuhRqtIml7aRUrFL8=,payment_method_id_other,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,Wed,Sat,Sep,Nov,2015,2015,52.0
24,qNpZJ7gsMPAFSvc5V3kgqtFfUQU2SZe7XSUYOAfqtoA=,payment_method_id_40,30,149,149,1,20151001,20151101,0,2015-10-01,2015-11-01,Thu,Sun,Oct,Nov,2015,2015,31.0
26,WnxdSgeK3Vg4BIaioJQYilnL+E1SDxq+PZIwmmfTwgM=,payment_method_id_34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015,31.0
27,ir1jh/vWZ932FpSC2WeMxdOHQ+OkuUo3ZPQs8Ms4J9g=,payment_method_id_34,0,0,149,1,20150930,20151031,0,2015-09-30,2015-10-31,Wed,Sat,Sep,Oct,2015,2015,31.0


Negative values in plan_duration imply the membership was canceled (is_canceled = 1).

In [14]:
transactions_df.loc[transactions_df['plan_duration'] < 0, 'plan_duration'] = 0

In [15]:
transactions_df['credit'] = (transactions_df['actual_amount_paid'] - transactions_df['plan_list_price'])
transactions_df['credit'].value_counts()

 0       12883088
 149       473082
-30        202774
-149        58159
 119        17611
 129        17348
 150         4127
-180         2002
-20          1959
-50          1155
 894          742
 134          623
 100          469
 1788         406
-120          151
 35           131
 131          114
 536           63
 480           48
 300           47
 799           45
 1599          40
 30            27
 450           13
 447           10
 930            9
 1000           9
 350            6
 500            5
 105            5
-108            4
-1              4
 400            2
 1150           2
 760            1
 50             1
 41             1
-699            1
 12             1
 1              1
Name: credit, dtype: int64

In [16]:
transactions_df['transaction_credit'] = np.where(transactions_df['credit'] == 0, 
                                                 'even', 
                                                 np.where(transactions_df['credit'] > 0, 
                                                          'credit', 'debit'))

In [17]:
import pickle

with open('Data/Processed/transactions_to_split.pickle', 'wb') as handle:
    pickle.dump(transactions_df, handle)